In [ ]:
#import packages used in this analysis
import pandas as pd
import numpy as np
import scipy as sp
import sys
sys.path.append('/Users/yuewang/Desktop/Desktop_Yue_MacBook_Pro/Nrf2_Project/')
import ccal

In [ ]:
#import TCGA TNBC gene expression data
read_file = pd.read_csv (r'/Volumes/SSD_Yue/TCGA_Data/brca_tcga_pan_can_atlas_2018/data_mrna_seq_v2_rsem.txt', sep='\t')
read_file = read_file[~read_file.Hugo_Symbol.isnull()]
read_file.index=read_file.Hugo_Symbol
read_file.drop(['Hugo_Symbol', 'Entrez_Gene_Id'],axis=1, inplace=True)
read_file=np.log2(read_file+1)
read_file=read_file.T
read_file.index=[i[0:12] for i in read_file.index]
read_file_2=read_file.groupby(read_file.columns.values,axis=1).mean()
basal_information=pd.read_csv('/Volumes/SSD_Yue/TCGA_Data/PAM50CLOW.csv')
list_basal=basal_information[basal_information.newSUBTYPE=='Basal'].iloc[:,0].tolist()

In [ ]:
df_exp_tnbc=read_file_2[read_file_2.index.isin(list_basal)]

In [ ]:
df_488=pd.read_csv('genes_separating_TCGA_TNBC_samples_comparing_against_BC_samples_12232022.csv')

In [ ]:
list_488=df_488.gene.tolist()

In [ ]:
#MYC gene is included to study how MYC heterogeneity is associated with other phenotypes in TNBC.
df_exp_tnbc=df_exp_tnbc[df_exp_tnbc.columns.intersection(list_488+['MYC'])]

In [ ]:
D = df_exp_tnbc.T
D1 = ((D - D.mean(axis=0))/D.std(axis=0))
D2  = 100 * ((D1 - D1.min(axis=0))/(D1.max(axis=0) - D1.min(axis=0)))
df_exp_tnbc_2 = D2

In [ ]:
#To perform NMF:
k_nmf= ccal.mf_consensus_cluster_with_ks(
    df_exp_tnbc_2,
    list(range(4,10)),
    n_job=4,
    plot_df=False,
    plot_w=True,
    plot_h=True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/'
#     n_clustering=1,
)

In [ ]:
#To perform hierarchical clustering on H and W matrices:
for k in range(4, 10): 
    w=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/w.tsv'.format(k), sep='\t', index_col=0)

    h=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/h.tsv'.format(k), sep='\t', index_col=0)

    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        w,
        list(range(4,10)),
        n_job = 1,
        distance_function = ccal.compute_correlation_distance,
        n_clustering = 40,
        plot_df = True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/W'.format(k))
    
    k_hcc = ccal.hierarchical_consensus_cluster_with_ks(
        h,
        list(range(4,10)),
        n_job = 1,
        distance_function = ccal.compute_correlation_distance,
        n_clustering = 40,
        plot_df = True,
    directory_path='/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/H'.format(k))

In [ ]:
#This color is consistent with the TCGA state colors in the paper figure
Onco_GPS_colors = [
                '#006837',  # F0 WNT signaling pathway
                '#FF00FF',  # F3 High-ELF5 & immune-active state
                '#603813',  # F5 MAGE protein family
                '#2E3192',  # F2 High-MYC & immune-suppressive state
                '#29ABE2',  # F1 Hypoxia
                '#cc99cc',  # F1-and-F2 mixture state Dark magenta 
                '#cc99cc',  # 3  
                '#fce94f',  # 4 Butter
                '#fe2c54',  # 6 Redish pink 
                '#AFF8DB',  # 7 Green
                '#ffad01',  # 8 Orange yellow

                '#AAAAFF',  # 10 Periwinkle

                '#FA8072',  # 12 Salmon
                '#ffccf9',  # 13 Pink 
                '#90EE90',  # 14 Light green 
                '#c4a000',  # 15
                '#e6194b',  # 16
                '#3cb44b',  # 17
                '#ffe119']  # 18

In [ ]:
#To generate identity map:
for k in range(4,10):
    
    w=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/w.tsv'.format(k), sep='\t', index_col=0)

    h=pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/h.tsv'.format(k), sep='\t', index_col=0)

    wT =w.T
    wT.index.name = h.index.name
    for s in range(4,10):
        try:
            h_clustering = pd.read_csv('/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/{}/H/{}/cluster_x_column.tsv'.format(k, s), sep='\t', index_col=0)
            h_element_labels = h_clustering.apply(lambda column: column.argmax())#.str.lstrip('C').astype(int)
            h_element_labels.value_counts()


            print("Making map with {} states and {} components...".format(s, k))
        
            gps_map = ccal.GPSMap(
                w = wT,
                h = h,
                w_n_pull = 3,
                w_pull_power = 3,
                h_n_pull = 3,
                h_pull_power = 3,
                plot = False,
                mds_random_seed = 27)

            gps_map.set_element_labels(
                        w_or_h = 'h',
                        element_labels = h_element_labels,
                        n_grid = 64,
                        label_colors = Onco_GPS_colors[0:s],
                        bandwidth_factor = 6)


            gps_map.plot_gps_map(
                w_or_h = 'h',
                element_marker_size = 24,
                title = 
                   'H Map {}-Components {}-States {}-Samples'.format(k, s, h.shape[1]),
                html_file_path = 
                   '/Volumes/SSD_Yue/NMF_Onco_gps_results/NMF_using_489_genes_selected_using_TCGA_breast_cancer_samples_including_MYC_on_185_TCGA_TNBC_samples_normalized_for_samples_12232022/maps/Onco-GPS_H_Map_{}-Comp_{}-Stat_{}-Samp.html'.format(k,k, 
                                                                                    s, 
                                                                                    len(h.columns)))
        except:
            continue
        